In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pre-trained model ResNet50 kullanılıyor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(400, 400, 3))

# İlk başta sadece son katmanları eğitilebilir yapıyoruz
base_model.trainable = False

# Model yapılandırması
inputs = tf.keras.Input(shape=(400, 400, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

# Modeli derleme
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Eğitim ve doğrulama verileri için ImageDataGenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rotation_range=40,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.5,
    zoom_range=0.3,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
valid_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
validation_generator = valid_datagen.flow(x_test, y_test, batch_size=32)

# İlk eğitim
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Fine-tuning: Sonraki katmanları da eğitilebilir hale getir
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Son 30 katmanı dondur
    layer.trainable = False

# Modeli tekrar derle (daha düşük bir öğrenme oranıyla)
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tuning sonrası eğitime devam et
fine_tune_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)
